<a href="https://colab.research.google.com/github/alexisdr/uned-tfg/blob/main/UNED-TFG-2-data-set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del Dataset

Durante este proceso se crea un dataset con lo datos procesados y listos para el entrenamiento del modelo.

# Parametros

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.
* dataset_path: ruta en el que se almacenará el dataset
* labels_file: ruta en la que se almacenará la lista de etiquetas

In [25]:
ruta_base = '/drive/My Drive/CorpusPFG/'

#Datos de origen
ruta_archivo_GoldStandardTrainingEA = ruta_base + 'GoldStandardTrainingEAs.txt'
ruta_archivo_GoldStandardTestEA = ruta_base + 'GoldStandardTestEAs.txt'
ruta_archivos_entrenamiento = ruta_base + 'Training/'
ruta_archivos_test = ruta_base + 'Test/'

#Dataset procesado
ruta_dataset = ruta_base + 'Dataset'

In [26]:
!pip install -q datasets unidecode nltk

Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [27]:
from google.colab import drive

drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


#Creación del dataset




Se crea un dataset con los tres segmentos: train, validation y test. Las etiquetas de train y validation se toman del fichero GoldStandardTrainingEAs.txt y las etiquetas del segmento test se toman del fichero GoldStandardTestEAs.txt.

Se cargan los archivos para su tratamiento.

In [28]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", 
                "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(
    ruta_archivo_GoldStandardTrainingEA, sep=' ', header=None, names=column_names)
df_goldStandardTestEAs = pd.read_csv(
    ruta_archivo_GoldStandardTestEA, sep=' ', header=None, names=column_names)

Generación de las etiquetas

In [29]:
df_goldStandardTrainingEAs

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27690409,T50.2X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24626366,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27967924,O90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25774939,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24878023,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14497,24368169,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14498,26850773,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14499,26850775,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14500,26850771,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
from datasets import ClassLabel

def crear_lista_etiquetas(dataframe):
  label_list = list(dataframe.TieneEA_EA1.unique())
  label_list.extend(list(dataframe.EA2.unique()))
  label_list.extend(list(dataframe.EA3.unique()))
  label_list.extend(list(dataframe.EA4.unique()))
  label_list.extend(list(dataframe.EA5.unique()))
  label_list.extend(list(dataframe.EA6.unique()))
  label_list.extend(list(dataframe.EA7.unique()))
  label_list.extend(list(dataframe.EA8.unique()))
  label_list.extend(list(dataframe.EA9.unique()))
  label_list.extend(list(dataframe.EA10.unique()))
  df = pd.DataFrame(label_list, columns=["label"])
  df = df.dropna()
  label_list = list(df.label.unique())
  return ClassLabel(num_classes=len(label_list), names=label_list)

class2label_entrenamiento = crear_lista_etiquetas(df_goldStandardTrainingEAs)
class2label_test = crear_lista_etiquetas(df_goldStandardTestEAs)

Precesamiento del texto de los informes

In [31]:
import json

str1 = "motivo de consulta"
str2 = "abadc add ad ad da da adsd"
str3 = "tratamiento"
str4 = "qwer qwrqwer eqwrqwer qwewqrq"

dictionary = {}

dictionary[str1] = str2
dictionary[str3] = str4

print(dictionary)

json_array = [ {'apartado' : k, 'texto' : dictionary[k]} for k in dictionary]
print(json.dumps(json_array))



{'motivo de consulta': 'abadc add ad ad da da adsd', 'tratamiento': 'qwer qwrqwer eqwrqwer qwewqrq'}
[{"apartado": "motivo de consulta", "texto": "abadc add ad ad da da adsd"}, {"apartado": "tratamiento", "texto": "qwer qwrqwer eqwrqwer qwewqrq"}]


In [32]:
import os
import re
from unidecode import unidecode
from nltk.corpus import stopwords 
import nltk
import json

JUICIO_CLINICO = 'juicio clinico'
JUICIO_DIAGNOSTICO = 'juicio diagnostico'
TRATAMIENTO = 'Tratamiento'
FIRMADO_XXX = '_FIRMADO_XXXX_'
ETIQUETAS = ["nif", "telefono", "movil", "fecha ingreso", "fecha alta", 
             "analitica", "comentarios", "juicio","clinico","tratamiento",
             "recomendaciones","exploraciones", "complementarias"]

re_sexo = re.compile(r'sexo [a-z]*')
re_juicio1 = re.compile(r'juicio [a-z]*')
re_juicio2 = re.compile(r'j[a-z]* clinico')
re_gestacion = re.compile(r'gestacion [a-z]*')

campos = ["nif", "telefono", "movil", "fecha ingreso", "fecha alta"]

apartados_de_interes = [
    "jdtco", "jc"] #, "sexo", "comentarios"
apartado_sexo = "sexo"

apartados_descartados = [
    "recomendaciones", "lopd", "pagina", "datos personales seran tratados"]    

TRAINING = 0
TEST = 1

nltk.download('stopwords')

def omitir_textos_anonimizacion (linea):
  # busca expresiones del tipo _HOSPITAL_XXXX_ _DIRECCION_XXXX_
  return re.sub (r'_\w+_XXXX_', '', linea)

def omitir_no_texto (linea):
  return re.sub (r'[^\w]', ' ', linea)

def omitir_numeros (linea):
  return re.sub(r'[0-9]+', ' ', linea)

def es_linea_comienzo_seleccion (linea):
  return normalizar_texto(linea.strip()) == JUICIO_CLINICO

def es_linea_fin_seleccion (linea):
  return FIRMADO_XXX in linea.strip()

def linea_tiene_contenido (linea):
  return linea.strip() != ''

def es_apartado_interes (linea, incluir_todo):
  apartados = apartados_de_interes
  if (incluir_todo):
    apartados.append(apartado_sexo)
  for apartado in apartados:
    if (apartado in linea):
      return True
  return False

def es_apartado (linea, incluir_todo):
  return (re_juicio1.match (linea) != None or
          re_juicio2.match (linea) != None or
          re_gestacion.match (linea) != None or
          es_apartado_interes (linea, incluir_todo))

def es_apartado_descartado (linea):
  for apartado in apartados_descartados:
    if (apartado in linea):
      return True
  return False

def detectar_etiquetas (linea):
  words = linea.strip().split()
  if len(words) >= 1 and len(words) <= 3:
      return linea
  return ""      

def omitir_stopwors (linea):
  stop_words = set(stopwords.words('spanish')) 
  palabras = linea.split() 
  resultado = ''
  for paralabra in palabras: 
      if not resultado in stop_words: 
        resultado += ' ' + paralabra
  return resultado

def normalizar_texto (linea):
  linea = unidecode(linea) 
  linea = linea.lower().strip()
  return linea

def quitar_etiquetas (linea):
  for etiqueta in ETIQUETAS:
    linea = linea.replace(etiqueta, "")
  return linea

def limpiar_texto (informe_medico, diccionario_apartados, incluir_todo):
  texto = ''
  seleccionar = False  
  apartado = ""
  texto_apartado = ""

  for lineaTexto in informe_medico:
    lineaTexto = omitir_textos_anonimizacion(lineaTexto)
    lineaTexto = normalizar_texto(lineaTexto)
    lineaTexto = omitir_no_texto(lineaTexto)
    lineaTexto = omitir_numeros(lineaTexto)
    if (not incluir_todo and apartado_sexo in lineaTexto):
      linea_array = lineaTexto.split(" ")      
      diccionario_apartados[apartado_sexo] = linea_array[len(linea_array)-1]
    elif (es_apartado(lineaTexto, incluir_todo)):
      if (apartado != "" and apartado != lineaTexto):
        diccionario_apartados[apartado] = texto_apartado
        texto_apartado = ""
      apartado = lineaTexto   
      seleccionar = True   
    elif (es_apartado_descartado(lineaTexto)):        
        if (texto_apartado != ""):
          diccionario_apartados[apartado] = texto_apartado
        apartado = "" 
        texto_apartado = ""
        seleccionar = False   
    if (seleccionar):      
      lineaTexto = omitir_stopwors(lineaTexto)  
      texto_apartado += lineaTexto 

  if (texto_apartado != ""):
    diccionario_apartados[apartado] = texto_apartado

  for apartado in diccionario_apartados.values():
    texto += apartado

  json_apartados = [ {'apartado' : k, 'texto' : diccionario_apartados[k]} for k in diccionario_apartados]
  return texto, json_apartados

def obtener_texto (acto, label_list, lista_nombres_informes_medicos, files_location, class2label):  
  texto_acto = ''  
  diccionario_apartados = {}
  for nombre_informe in lista_nombres_informes_medicos:
    with open (files_location + nombre_informe, 'rt') as informe_medico:
      resultado = limpiar_texto(informe_medico, diccionario_apartados, False)
    if (resultado[0].strip() == ""):
      with open (files_location + nombre_informe, 'rt') as informe_medico:
        resultado = limpiar_texto(informe_medico, diccionario_apartados, True)
    texto_acto = texto_acto + resultado[0]
    texto_acto = texto_acto + "\n"
  label_list_int = convertir_codigos_EA_a_entero(class2label, label_list)
  return [acto,
          label_list_int[0], 
          label_list[0], 
          label_list_int,
          label_list, 
          lista_nombres_informes_medicos, 
          texto_acto, 
          json.dumps(resultado[1])]

def convertir_codigos_EA_a_entero(class2label, label_list):
  label_list_int = list()
  for codigo_ea in label_list:
    label_list_int.append(class2label.str2int(codigo_ea))
  return label_list_int

def tomar_codigos_EA_de_todas_las_columnas(row):
  label_list = list()
  for col in column_names:
    if (col == column_names[0]):
      continue
    if (type(row[col]) == str):
      label_list.append(row[col].strip())
  return label_list

def crear_vector_datos (dataframe, file_list, files_location, class2label, cantidad): 
  #dataframe_SinNone = dataframe.query("TieneEA_EA1 != 'NONE'")
  #dataframe_SinNone = dataframe_SinNone.reset_index()  
  ds_resultado = []
  i = 0
  for index, row in dataframe.iterrows():
    texto_filtro = str(row['Acto']) + '-'
    archivos_acto = filter(lambda x: texto_filtro in x, file_list)       
    ds_resultado.append(obtener_texto(row['Acto'], tomar_codigos_EA_de_todas_las_columnas(row), 
                                      list(archivos_acto), files_location, class2label))
    i += 1
    if (cantidad > 0 and i > cantidad):
     break
  return ds_resultado

def crear_dataframe (labeled_array):
  return pd.DataFrame(labeled_array, 
                      columns=['acto', 
                               'label', 'label_str', 
                               'label_list', 'label_list_str', 
                               'informes', 
                               'text', 'json'])

def crear_dataframe_para_un_informe (nombre_archivo_informe):
  acto = nombre_archivo_informe.split("-")[0]
  labeled_array_training = crear_vector_datos (
      df_goldStandardTrainingEAs.query("Acto == " + acto), [nombre_archivo_informe], 
        ruta_archivos_entrenamiento, class2label_entrenamiento, 0)
  return crear_dataframe (labeled_array_training)  

def crear_dataframe_train_test ():
  lista_archivos_entrenamiento = [f for f in os.listdir(ruta_archivos_entrenamiento) 
                      if os.path.isfile(os.path.join(ruta_archivos_entrenamiento, f))]
  vector_datos_entrenamiento = crear_vector_datos (
      df_goldStandardTrainingEAs, lista_archivos_entrenamiento, 
      ruta_archivos_entrenamiento, class2label_entrenamiento, 0)
  df_entrenamiento = crear_dataframe (vector_datos_entrenamiento)
  
  lista_datos_test = [f for f in os.listdir(ruta_archivos_test) 
                  if os.path.isfile(os.path.join(ruta_archivos_test, f))]
  vector_datos_test = crear_vector_datos (
      df_goldStandardTestEAs, lista_datos_test, ruta_archivos_test, class2label_test, 0)
  df_test = crear_dataframe (vector_datos_test)

  return df_entrenamiento, df_test



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
crear_dataframe_para_un_informe("23062488-162179798.txt")

,acto,label,label_str,label_list,label_list_str,informes,text,json
0,23062488,33,T85.71XA,"[33, 184]","[T85.71XA, T82.898A]",[23062488-162179798.txt],hombre juicio clinico exitus sepsis de origen ...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."


In [34]:
dataframes = crear_dataframe_train_test()

In [35]:
dataframes[0]

,acto,label,label_str,label_list,label_list_str,informes,text,json
0,27690409,0,T50.2X5A,[0],[T50.2X5A],[27690409-171690541.txt],hombre juicio clinico primer episodio de desco...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."
1,24626366,1,NONE,[1],[NONE],[24626366-163054389.txt],mujer juicio clinico neumonia probablemente ne...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."
2,27967924,2,O90.0,[2],[O90.0],[27967924-172443607.txt],gestacion actual f u r f p p gestacion unica ...,"[{""apartado"": ""gestacion actual"", ""texto"": "" g..."
3,25774939,1,NONE,[1],[NONE],[25774939-166138278.txt],hombre juicio clinico episodio de icc leve pre...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."
4,24878023,1,NONE,[1],[NONE],[24878023-163476964.txt],hombre juicio clinico celulitis perirobitaria ...,"[{""apartado"": ""sexo"", ""texto"": ""hombre""}, {""ap..."
...,...,...,...,...,...,...,...,...
14497,24368169,1,NONE,[1],[NONE],[24368169-162102373.txt],mujer juicio clinico anemizacion aguda sobre a...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."
14498,26850773,1,NONE,[1],[NONE],"[26850773-169269452.txt, 26850773-169270850.txt]",juicio clinico cancer de mama derecha anteced...,"[{""apartado"": ""juicio clinico"", ""texto"": "" jui..."
14499,26850775,1,NONE,[1],[NONE],[26850775-169239813.txt],mujer juicio clinico intolerancia a aines asma...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."
14500,26850771,1,NONE,[1],[NONE],[26850771-169251366.txt],mujer juicio diagnostico multiples polipos col...,"[{""apartado"": ""sexo"", ""texto"": ""mujer""}, {""apa..."


Composición del dataset

In [40]:
from datasets import Dataset, Value, DatasetDict

def set_dataset_features (dataset, class2label):
  new_features = dataset.features.copy()
  new_features["label"] = class2label
  return dataset.cast(new_features)
  
dataset_entrenamiento = Dataset.from_pandas(dataframes[TRAINING])
dataset_entrenamiento = set_dataset_features (dataset_entrenamiento, class2label_entrenamiento)

dataset_entrenamiento_validacion = dataset_entrenamiento.train_test_split(test_size=0.1)

dataset_test = Dataset.from_pandas(dataframes[TEST])
dataset_test = set_dataset_features (dataset_test, class2label_test)

dataset_completo = DatasetDict(
    {
        "train": dataset_entrenamiento_validacion["train"],
        "validation": dataset_entrenamiento_validacion["test"],
        "test": dataset_test,
    })

dataset_completo

Casting the dataset:   0%|          | 0/14502 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3629 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 13051
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 1451
    })
    test: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 3629
    })
})

In [41]:
dataset_completo["train"][0]

{'acto': 28371702,
 'label': 1,
 'label_str': 'NONE',
 'label_list': [1],
 'label_list_str': ['NONE'],
 'informes': ['28371702-173743355.txt'],
 'text': 'hombre juicio clinico bronquitis aguda reagudizacion de epoc aislamiento de pseudomonas en esputo insuficiencia cardiaca descompensada fibrilacion auricular no respuesta ventricular controlada no conocida insuficiencia respiratoria global aguda en paciente con insuficiencia respiratoria de base acidosis respiratoria fracaso renal agudo sobre enfermedad renal cronica probablemente prerrenal hiperpotasemia secundaria a fra y acidosis mioclonias multifactoriales sondaje vesical retirado al alta los previos namc ex fumador hta dm tipo dislipemia hiperuricemia cardiopatia hipertensiva e isquemica enfermedad coronaria severa de vasos portador de stent epoc erc anemia cronica infarto hemorragico en acp derecha embolico sin fuente conocida en hemicolectomia derecha ampliada laparoscopica por ca de colon stui moderado severo probable hbp muy n

In [42]:
dataset_completo["train"][0]['text']

'hombre juicio clinico bronquitis aguda reagudizacion de epoc aislamiento de pseudomonas en esputo insuficiencia cardiaca descompensada fibrilacion auricular no respuesta ventricular controlada no conocida insuficiencia respiratoria global aguda en paciente con insuficiencia respiratoria de base acidosis respiratoria fracaso renal agudo sobre enfermedad renal cronica probablemente prerrenal hiperpotasemia secundaria a fra y acidosis mioclonias multifactoriales sondaje vesical retirado al alta los previos namc ex fumador hta dm tipo dislipemia hiperuricemia cardiopatia hipertensiva e isquemica enfermedad coronaria severa de vasos portador de stent epoc erc anemia cronica infarto hemorragico en acp derecha embolico sin fuente conocida en hemicolectomia derecha ampliada laparoscopica por ca de colon stui moderado severo probable hbp muy numerosos ingresos por agudizacion de epoc insuficiencia cardiaca e insuficiencia respiratoria con alteracion funcion renal en ocasiones itu tratamiento a

In [43]:
dataset_completo["train"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['T50.2X5A', 'NONE', 'O90.0', 'T46.5X5A', 'E89.0', 'H59.022', 'Y84.2', 'T83.021A', 'K94.23', 'T50.8X5A', 'Y95', 'T50.905A', 'T50.0X5A', 'L27.0', 'T85.398A', 'T38.0X5A', 'T85.79XA', 'P39.1', 'T84.498A', 'T84.82XA', 'K91.841', 'I97.618', 'T80.1XXA', 'T84.89XA', 'T40.605A', 'R50.82', 'T81.4XXA', 'T38.3X5A', 'G97.1', 'O75.2', 'N99.820', 'T45.1X5A', 'T39.1X5A', 'T85.71XA', 'L76.02', 'K66.0', 'Y83.1', 'T82.120A', 'K12.31', 'P01.1', 'G25.1', 'G89.18', 'T38.0X5D', 'L76.31', 'M96.830', 'G97.41', 'T47.4X5A', 'N99.821', 'T45.515A', 'T80.211A', 'T46.2X5S', 'P36.9', 'T36.0X5A', 'T84.84XA', 'G62.0', 'D70.1', 'T82.868A', 'T79.6XXA', 'T40.2X5A', 'T39.395A', 'T41.5X5A', 'T84.223A', 'T45.1X5D', 'T83.498A', 'L76.22', 'T43.295A', 'K94.12', 'P39.8', 'O86.29', 'K91.71', 'T84.021A', 'M96.840', 'Y83.2', 'T87.81', 'I97.130', 'T46.0X5A', 'T84.51XA', 'P03.4', 'Y64.0', 'J95.811', 'T39.2X5A', 'G97.51', 'T48.6X5A', 'T45.615A', 'H59.021', 'T82.7XXA',

In [44]:
dataset_completo["test"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['NONE', 'T38.0X5A', 'Y83.1', 'P02.69', 'T45.515A', 'T36.95XA', 'T50.2X5A', 'T39.315A', 'T39.95XA', 'O91.22', 'T86.10', 'Y95', 'T39.395A', 'T84.54XA', 'T84.226A', 'P01.1', 'K91.840', 'J95.811', 'T82.7XXA', 'T43.505A', 'T50.8X5A', 'E89.0', 'T45.1X5A', 'K94.29', 'T81.83XA', 'T45.615A', 'T86.5', 'K91.841', 'T88.8XXA', 'T80.1XXA', 'L76.22', 'O75.2', 'T46.0X5A', 'I97.190', 'O04.5', 'T81.4XXA', 'I95.2', 'T88.59XA', 'T36.1X5A', 'M96.1', 'T42.8X5A', 'O75.82', 'I97.121', 'T83.51XA', 'Y83.6', 'K66.0', 'T46.2X5A', 'Y84.2', 'L76.32', 'P03.89', 'K91.71', 'T84.428A', 'T85.49XA', 'T50.905A', 'G72.0', 'T83.89XA', 'T86.11', 'T42.4X5A', 'P03.4', 'K94.09', 'T80.212A', 'T39.2X5A', 'Y83.8', 'T39.4X5A', 'K12.31', 'P39.3', 'T82.09XD', 'K91.89', 'T82.9XXA', 'T82.868A', 'T42.75XA', 'G89.18', 'E89.89', 'R50.82', 'T50.3X5A', 'T40.2X5A', 'T82.524A', 'P13.4', 'T83.59XA', 'P02.7', 'T82.855D', 'T50.1X5A', 'P15.8', 'I97.51', 'T45.525A', 'N99.111', 'K9

In [45]:
dataset_completo.save_to_disk(ruta_dataset)

Saving the dataset (0/1 shards):   0%|          | 0/13051 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1451 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3629 [00:00<?, ? examples/s]